In [1]:
import pandas as pd
import numpy as np

In [1]:
"""
    https://data.cityofnewyork.us/Public-Safety/EMS-Incident-Dispatch-Data/76xm-jjuj/explore/query/
    SELECT%0A%20%20%60cad_incident_id
    %60%2C%0A%20%20%60incident_datetime
    %60%2C%0A%20%20%60incident_travel_tm_seconds_qy
    %60%2C%0A%20%20%60zipcode
    %60%0AORDER%20BY%20%60incident_datetime
    %60%20DESC%20NULL%20FIRST/page/aggregate
"""

'\n    https://data.cityofnewyork.us/Public-Safety/EMS-Incident-Dispatch-Data/76xm-jjuj/explore/query/\n    SELECT%0A%20%20%60cad_incident_id\n    %60%2C%0A%20%20%60incident_datetime\n    %60%2C%0A%20%20%60incident_travel_tm_seconds_qy\n    %60%2C%0A%20%20%60zipcode\n    %60%0AORDER%20BY%20%60incident_datetime\n    %60%20DESC%20NULL%20FIRST/page/aggregate\n'

## Load & Parse EMS Data

In [2]:
ems_src = "https://data.cityofnewyork.us/resource/76xm-jjuj.json?$limit=243979&%24where=cad_incident_id%20%25%2095%20%3D%200"
ems_df = pd.read_json(ems_src)
print(ems_df.columns)
ems_df

Index(['cad_incident_id', 'incident_datetime', 'initial_call_type',
       'initial_severity_level_code', 'final_call_type',
       'final_severity_level_code', 'first_assignment_datetime',
       'valid_dispatch_rspns_time_indc', 'dispatch_response_seconds_qy',
       'first_activation_datetime', 'first_on_scene_datetime',
       'valid_incident_rspns_time_indc', 'incident_response_seconds_qy',
       'incident_travel_tm_seconds_qy', 'first_to_hosp_datetime',
       'first_hosp_arrival_datetime', 'incident_close_datetime',
       'held_indicator', 'incident_disposition_code', 'borough',
       'incident_dispatch_area', 'zipcode', 'policeprecinct',
       'citycouncildistrict', 'communitydistrict', 'communityschooldistrict',
       'congressionaldistrict', 'reopen_indicator', 'special_event_indicator',
       'standby_indicator', 'transfer_indicator'],
      dtype='object')


,cad_incident_id,incident_datetime,initial_call_type,initial_severity_level_code,final_call_type,final_severity_level_code,first_assignment_datetime,valid_dispatch_rspns_time_indc,dispatch_response_seconds_qy,first_activation_datetime,...,zipcode,policeprecinct,citycouncildistrict,communitydistrict,communityschooldistrict,congressionaldistrict,reopen_indicator,special_event_indicator,standby_indicator,transfer_indicator
0,223655175,2022-12-31T23:59:32.000,CARD,3,CARD,3,2023-01-01T00:03:16.000,Y,224,2023-01-01T00:03:41.000,...,10457,46.0,15.0,205.0,9.0,15.0,N,N,N,N
1,223655080,2022-12-31T23:39:30.000,SICK,6,SICK,6,2022-12-31T23:53:50.000,Y,860,2022-12-31T23:53:57.000,...,11692,100.0,31.0,414.0,27.0,5.0,N,N,N,N
2,223654985,2022-12-31T23:22:05.000,STATEP,2,STATEP,2,2022-12-31T23:24:39.000,Y,154,2022-12-31T23:24:51.000,...,10039,32.0,9.0,110.0,5.0,13.0,N,N,N,N
3,223654795,2022-12-31T22:37:50.000,EDP,7,EDP,7,NaN,N,0,NaN,...,10028,19.0,5.0,108.0,2.0,12.0,N,N,N,N
4,223654700,2022-12-31T22:16:14.000,ALTMFC,3,ALTMFC,3,2022-12-31T22:16:26.000,Y,12,2022-12-31T22:17:07.000,...,10039,32.0,9.0,110.0,5.0,13.0,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243974,60310180,2006-01-31T01:34:40.000,SICK,6,SICK,6,2006-01-31T01:35:15.000,Y,35,2006-01-31T01:35:28.000,...,11225,71.0,40.0,309.0,17.0,9.0,N,N,N,N
243975,60303625,2006-01-30T23:30:13.000,UNC,2,UNC,2,2006-01-30T23:30:50.000,Y,37,2006-01-30T23:34:10.000,...,10458,48.0,15.0,206.0,10.0,15.0,N,N,N,N
243976,60303530,2006-01-30T22:42:12.000,DRUG,4,DRUG,4,2006-01-30T22:42:57.000,Y,45,2006-01-30T22:45:02.000,...,10009,13.0,4.0,106.0,2.0,12.0,N,N,N,N
243977,60303435,2006-01-30T21:57:23.000,INHALE,5,INHALE,5,2006-01-30T21:57:40.000,Y,17,2006-01-30T21:57:47.000,...,11212,73.0,42.0,316.0,23.0,8.0,N,N,N,N


In [3]:
# adding basic filters
target_cols = ["incident_travel_tm_seconds_qy", "incident_datetime", "zipcode"]
df = ems_df[target_cols].dropna()
df = df.rename(columns={"incident_travel_tm_seconds_qy": "travel_time"})

# specifying types
df["zipcode"] = df["zipcode"].astype("category")
df["travel_time"] = pd.to_numeric(df["travel_time"])
df["incident_datetime"] = pd.to_datetime(df["incident_datetime"])

# splitting datetime
df["date"] = pd.to_datetime(df["incident_datetime"]).dt.date
df["time"] = pd.to_datetime(df["incident_datetime"]).dt.time
df["time_block"] = df["time"].apply(lambda x: x.hour)
df = df.drop(columns=["incident_datetime"])
df

,travel_time,zipcode,date,time,time_block
0,575.0,10457,2022-12-31,23:59:32,23
1,316.0,11692,2022-12-31,23:39:30,23
2,521.0,10039,2022-12-31,23:22:05,23
4,668.0,10039,2022-12-31,22:16:14,22
5,242.0,10456,2022-12-31,21:50:28,21
...,...,...,...,...,...
243974,298.0,11225,2006-01-31,01:34:40,1
243975,253.0,10458,2006-01-30,23:30:13,23
243976,422.0,10009,2006-01-30,22:42:12,22
243977,93.0,11212,2006-01-30,21:57:23,21


## Load and Parse Traffic Data

In [4]:
traffic_src = "https://data.cityofnewyork.us/resource/btm5-ppia.json?$limit=42800"
traffic_df = pd.read_json(traffic_src)
traffic_df["date"] = pd.to_datetime(traffic_df["date"])
print(traffic_df.columns)
traffic_df

Index(['id', 'segmentid', 'roadway_name', 'from', 'to', 'direction', 'date',
       '_12_00_1_00_am', '_1_00_2_00am', '_2_00_3_00am', '_3_00_4_00am',
       '_4_00_5_00am', '_5_00_6_00am', '_6_00_7_00am', '_7_00_8_00am',
       '_8_00_9_00am', '_9_00_10_00am', '_10_00_11_00am', '_11_00_12_00pm',
       '_12_00_1_00pm', '_1_00_2_00pm', '_2_00_3_00pm', '_3_00_4_00pm',
       '_4_00_5_00pm', '_5_00_6_00pm', '_6_00_7_00pm', '_7_00_8_00pm',
       '_8_00_9_00pm', '_9_00_10_00pm', '_10_00_11_00pm', '_11_00_12_00am'],
      dtype='object')


,id,segmentid,roadway_name,from,to,direction,date,_12_00_1_00_am,_1_00_2_00am,_2_00_3_00am,...,_2_00_3_00pm,_3_00_4_00pm,_4_00_5_00pm,_5_00_6_00pm,_6_00_7_00pm,_7_00_8_00pm,_8_00_9_00pm,_9_00_10_00pm,_10_00_11_00pm,_11_00_12_00am
0,1,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,2012-01-09,20.0,10.0,11.0,...,104.0,105.0,147.0,120.0,91.0,83.0,74.0,49.0,42.0,42.0
1,2,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,2012-01-10,21.0,16.0,8.0,...,102.0,98.0,133.0,131.0,95.0,73.0,70.0,63.0,42.0,35.0
2,3,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,2012-01-11,27.0,14.0,6.0,...,115.0,115.0,130.0,143.0,106.0,89.0,68.0,64.0,56.0,43.0
3,4,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,2012-01-12,22.0,7.0,7.0,...,71.0,127.0,122.0,144.0,122.0,76.0,64.0,58.0,64.0,43.0
4,5,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,2012-01-13,31.0,17.0,7.0,...,113.0,126.0,133.0,135.0,102.0,106.0,58.0,58.0,55.0,54.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42751,373,35832,WEST 49th STREET,Rockefeller Plaza,6th Avenue,WB,2020-11-18,68.0,63.0,31.0,...,152.0,177.0,162.0,164.0,183.0,173.0,143.0,113.0,127.0,133.0
42752,373,35832,WEST 49th STREET,Rockefeller Plaza,6th Avenue,WB,2020-11-19,71.0,59.0,42.0,...,166.0,162.0,187.0,182.0,180.0,190.0,162.0,155.0,234.0,166.0
42753,373,35832,WEST 49th STREET,Rockefeller Plaza,6th Avenue,WB,2020-11-20,111.0,79.0,54.0,...,157.0,186.0,170.0,158.0,194.0,196.0,222.0,174.0,182.0,178.0
42754,373,35832,WEST 49th STREET,Rockefeller Plaza,6th Avenue,WB,2020-11-21,129.0,95.0,63.0,...,139.0,147.0,153.0,183.0,163.0,184.0,183.0,169.0,187.0,211.0


In [5]:
new_cols = ["time_block", "date", "zipcode", "traffic_volume"]
new_data = dict([[name, []] for name in new_cols])
for i, row in traffic_df.iterrows():
    for j in range(7,len(row)):  # 7 to 30
        new_data["date"].append(row[6])
#         location_data = [name.lower().strip() for name in row[2:5]]
        new_data["zipcode"].append(row[2].lower().strip())
        
        new_data["traffic_volume"].append(row[j])
        new_data["time_block"].append(j-7)

In [6]:
tvol_df = pd.DataFrame(new_data)
tvol_df

,time_block,date,zipcode,traffic_volume
0,0,2012-01-09,beach street,20.0
1,1,2012-01-09,beach street,10.0
2,2,2012-01-09,beach street,11.0
3,3,2012-01-09,beach street,14.0
4,4,2012-01-09,beach street,13.0
...,...,...,...,...
1026139,19,2020-11-22,west 49th street,177.0
1026140,20,2020-11-22,west 49th street,160.0
1026141,21,2020-11-22,west 49th street,162.0
1026142,22,2020-11-22,west 49th street,147.0


In [7]:
# save streets for testing street to zip
# example_streets = pd.DataFrame(list(tvol_df["zipcode"].unique()), columns=["streets"])
# example_streets["zipcodes"] = [""]*len(example_streets)
# street_loc = "data/example_streets.csv"
# example_streets.to_csv(street_loc, index=False)  # np.savetxt(street_loc, example_streets, delimiter=",", fmt='%s')
# print("streets saved for later at:", street_loc)
# example_streets

In [12]:
# Dependency: https://github.com/ipython/ipynb
from ipynb.fs.full.streetzip import street_to_zip

zips = list(tvol_df["zipcode"].unique())
translated_zipcodes = street_to_zip(zips, True)

database done loading, translating streets in progress

Could not find little clove road
Could not find clove rd
Could not find hylan blvd
Could not find manor rd
Could not find todt hill rd
Could not find malcolm x blvd
Could not find marcus garvey blvd
Could not find veterans avenue
Could not find richmond rd
Could not find amboy rd
Could not find 4th ave
Could not find shore blvd
Could not find shore pkwy north
Could not find shore pkwy south
Could not find shore rd
Could not find shore pkwy
Could not find ave h
Could not find so conduit ave
Could not find east 174 street
Could not find murdock ave nue
Could not find seagirt blvd
Could not find east 167 street
Could not find east 170 street
Could not find east 188 street
Could not find east 165 street
Could not find east 169 street
Could not find east 181 street
Could not find east 134 street
Could not find cross island pkwy sr south
Could not find conner street
Could not find east 138 street
Could not find east 149 street
Could not

Could not find 7th avenue
Could not find 8th avenue
Could not find 82nd street
Could not find 9th avenue
Could not find 94th avenue
Could not find 6th avenue
Could not find colonial rd
Could not find east 135th street
Could not find east 138th street
Could not find east 144th street
Could not find east 149th street
Could not find east 161st street
Could not find east 163rd street
Could not find east 169th street
Could not find east 170th street
Could not find east 174th street
Could not find east 177th street
Could not find east 18th street
Could not find e 204th st
Could not find east 23rd street
Could not find east 233rd street
Could not find east 241st street
Could not find honeywell st
Could not find pelham bridge rd
Could not find queens blvd
Could not find southern blvd
Could not find st. anns ave
Could not find west 125th street
Could not find west 135th street
Could not find west 145th street
Could not find west 155th street
Could not find west 167th street
Could not find west 

In [13]:
zip_lookup = dict([[zips[i], translated_zipcodes[i]] for i in range(len(zips))])
zip_lookup

{'beach street': 10304.0,
 'little clove road': '',
 'narrows road south': 10305.0,
 'ocean terrace': 10301.0,
 'bay st': 10304.0,
 'clove rd': '',
 'hylan blvd': '',
 'manor rd': '',
 'targee st': 10304.0,
 'todt hill rd': '',
 'van duzer street': 10304.0,
 'victory boulevard': 10301.0,
 'morgan ave': 11211.0,
 'park avenue': 10016.0,
 '8 ave': 10026.0,
 'kingsland avenue': 11222.0,
 'lewis ave': 11221.0,
 'humboldt street': 11211.0,
 'kent avenue': 11249.0,
 'ralph ave': 11234.0,
 'rockaway ave': 11233.0,
 'stuyvesant avenue': 11221.0,
 'throop avenue': 10469.0,
 'union avenue': 11211.0,
 'malcolm x blvd': '',
 'graham avenue': 11222.0,
 'greene avenue': 11238.0,
 'greenpoint avenue': 11101.0,
 'kingston ave': 11213.0,
 '3 ave': 11420.0,
 '4 ave': 11435.0,
 '5 ave': 11364.0,
 '6 ave': 11215.0,
 '7 ave': 11377.0,
 'new york ave': 11210.0,
 'quincy st': 11238.0,
 'smith st': 11231.0,
 'tompkins ave': 11206.0,
 'union st': 11213.0,
 'van brunt st': 11231.0,
 'vanderbilt ave': 11238.0,
 

In [14]:
# finalize translation
tvol_df["zipcode"] = tvol_df["zipcode"].apply(lambda x: zip_lookup[x])
tvol_df = tvol_df[tvol_df["zipcode"] != '']  # prune zipcodes that weren't found
tvol_df

,time_block,date,zipcode,traffic_volume
0,0,2012-01-09,10304.0,20.0
1,1,2012-01-09,10304.0,10.0
2,2,2012-01-09,10304.0,11.0
3,3,2012-01-09,10304.0,14.0
4,4,2012-01-09,10304.0,13.0
...,...,...,...,...
1025923,19,2020-10-25,10451.0,178.0
1025924,20,2020-10-25,10451.0,127.0
1025925,21,2020-10-25,10451.0,110.0
1025926,22,2020-10-25,10451.0,87.0


In [40]:
# fix zipcode to int
tvol_df["zipcode"] = pd.to_numeric(tvol_df["zipcode"].apply(lambda x:int(x)))

# standardize datetime stuff
tvol_df["date"] = pd.to_datetime(tvol_df["date"]).dt.date

tvol_df

/var/folders/mv/k0krwyts3z5cfn3y2tltw0sc0000gn/T/ipykernel_86332/487224960.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tvol_df["zipcode"] = pd.to_numeric(tvol_df["zipcode"].apply(lambda x:int(x)))
/var/folders/mv/k0krwyts3z5cfn3y2tltw0sc0000gn/T/ipykernel_86332/487224960.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tvol_df["date"] = pd.to_datetime(tvol_df["date"]).dt.date


,time_block,date,zipcode,traffic_volume
0,0,2012-01-09,10304,20.0
1,1,2012-01-09,10304,10.0
2,2,2012-01-09,10304,11.0
3,3,2012-01-09,10304,14.0
4,4,2012-01-09,10304,13.0
...,...,...,...,...
1025923,19,2020-10-25,10451,178.0
1025924,20,2020-10-25,10451,127.0
1025925,21,2020-10-25,10451,110.0
1025926,22,2020-10-25,10451,87.0


In [70]:
# standardizing zipcode
df["zipcode"] = df["zipcode"].apply(lambda x: int(x.replace(',', '')))
df["zipcode"] = df["zipcode"].astype(int)

# reminder of ems data
df

,travel_time,zipcode,date,time,time_block
0,575.0,10457,2022-12-31,23:59:32,23
1,316.0,11692,2022-12-31,23:39:30,23
2,521.0,10039,2022-12-31,23:22:05,23
4,668.0,10039,2022-12-31,22:16:14,22
5,242.0,10456,2022-12-31,21:50:28,21
...,...,...,...,...,...
243974,298.0,11225,2006-01-31,01:34:40,1
243975,253.0,10458,2006-01-30,23:30:13,23
243976,422.0,10009,2006-01-30,22:42:12,22
243977,93.0,11212,2006-01-30,21:57:23,21


In [125]:
example = tvol_df.sample().iloc[0]
example

time_block                10
date              2020-10-17
zipcode                11234
traffic_volume          91.0
Name: 960226, dtype: object

In [126]:
df[(df["date"] == example["date"]) & (df["zipcode"] == example["zipcode"])]

,travel_time,zipcode,date,time,time_block
35288,850.0,11234,2020-10-17,14:43:01,14
35295,491.0,11234,2020-10-17,11:11:37,11


In [113]:
# Note: use datatypes to confirm that 
df.dtypes

travel_time    float64
zipcode          int64
date            object
time            object
time_block       int64
dtype: object

In [114]:
tvol_df.dtypes

time_block          int64
date               object
zipcode             int64
traffic_volume    float64
dtype: object

## Merging the Dataset

In [ ]:
from tqdm import tqdm  # progress bar since this is a big database operation

found_counter = 0
notfound_counter = 0

merge_columns = ["time_block", "date", "zipcode", "traffic_volume", "avg_travel_time"]
merge_data = []
for i, row in tqdm(tvol_df.iterrows(), total=tvol_df.shape[0]):  # TODO: fix date-time comparison
    related_ems = df[(df["time_block"] == row["time_block"]) &
                       (df["date"] == row["date"]) &
                       (df["zipcode"] == row["zipcode"])]
    avg_travel = pd.NA
    if len(related_ems) > 0:
        avg_travel = related_ems["travel_time"].mean()
        found_counter += 1
        print("Travel time found at:", row)  # this means its working and we are finding matches
    else:
        notfound_counter += 1
    add_data = [row["time_block"], row["date"], row["zipcode"], row["traffic_volume"], avg_travel]
    merge_data.append(add_data)
merge_data  # 4% per hour?

  0%|                                         | 559/850800 [00:09<3:48:43, 61.95it/s]

Travel time found at: time_block                22
date              2012-01-13
zipcode                10301
traffic_volume          96.0
Name: 838, dtype: object
Travel time found at: time_block                 5
date              2012-01-14
zipcode                10301
traffic_volume          12.0
Name: 845, dtype: object


  0%|                                         | 706/850800 [00:11<3:51:58, 61.08it/s]

Travel time found at: time_block                22
date              2012-01-13
zipcode                10301
traffic_volume          48.0
Name: 982, dtype: object
Travel time found at: time_block                 5
date              2012-01-14
zipcode                10301
traffic_volume           8.0
Name: 989, dtype: object


  0%|                                        | 1567/850800 [00:25<3:49:29, 61.67it/s]

...

 63%|██████████████████████▌             | 533101/850800 [2:36:23<1:03:27, 83.44it/s]

Travel time found at: time_block                 1
date              2017-10-06
zipcode                10029
traffic_volume         603.0
Name: 642889, dtype: object


 63%|██████████████████████▌             | 533625/850800 [2:36:29<1:05:37, 80.56it/s]

Travel time found at: time_block                22
date              2017-10-06
zipcode                10002
traffic_volume          40.0
Name: 643558, dtype: object


 63%|██████████████████████▌             | 533642/850800 [2:36:30<1:15:25, 70.08it/s]

In [ ]:
# validate the percentage of the database that was successfully merged
found_perc = found_counter / (found_counter + notfound_counter)
print("Found ratio:", found_counter, "of", (found_counter + notfound_counter), "records")
print(found_perc*100, "%")

In [ ]:
merged_df = pd.DataFrame(merge_data, columns=merge_columns)
merged_df

In [ ]:
# confirm that some of the entries were written
merged_df[~merged_df["avg_travel_time"].isna()]

In [ ]:
merge_loc = "data/new_merged_data.csv"
merged_df.to_csv(merge_loc, index=False)
print("streets saved for later at:", merge_loc)